In [2]:
import numpy as np
import librosa
import librosa.display as display
import matplotlib.pyplot as plt
import os
import h5py
import time
from IPython.display import Audio
from config import PARAS
from mel_dealer import mel_converter



MIX_PATH = '../DSD100/Mixtures'
SRC_PATH = '../DSD100/Sources'

# Get File Paths

- Path Dictionary
    - Music File Index
        - Track 1 -> path
        - Track 2 -> path

In [3]:
dev_file_paths = dict()
test_file_paths = dict()

for (dirpath, dirnames, filenames) in os.walk(MIX_PATH):
    if not dirnames:
        if 'Test' in dirpath:
            file_name = dirpath.split('Test/')[-1].split('-')[0].strip()
            test_file_paths[file_name] = dict()
            test_file_paths[file_name]['mix'] = dirpath + '/' + filenames[0]
        elif 'Dev' in dirpath:
            file_name = dirpath.split('Dev/')[-1].split('-')[0].strip()
            dev_file_paths[file_name] = dict()
            dev_file_paths[file_name]['mix'] = dirpath + '/' + filenames[0]

for (dirpath, dirnames, filenames) in os.walk(SRC_PATH):
    if not dirnames:
        if 'Test' in dirpath:
            file_name = dirpath.split('Test/')[-1].split('-')[0].strip()
            for trackname in filenames:
                track = trackname.split('.wav')[0]
                test_file_paths[file_name][track] = dirpath + '/' + trackname
        elif 'Dev' in dirpath:
            file_name = dirpath.split('Dev/')[-1].split('-')[0].strip()
            for trackname in filenames:
                track = trackname.split('.wav')[0]
                dev_file_paths[file_name][track] = dirpath + '/' + trackname
    

# Build Feature Extractor

In [4]:
PARAS.SR, PARAS.N_FFT, PARAS.N_MEL, PARAS.SAMPLE_TIME

(16000, 512, 128, 1)

In [5]:
test_path = dev_file_paths['056']
test_path, len(dev_file_paths)

({'mix': '../DSD100/Mixtures/Dev/056 - Arise - Run Run Run/mixture.wav',
  'vocals': '../DSD100/Sources/Dev/056 - Arise - Run Run Run/vocals.wav',
  'drums': '../DSD100/Sources/Dev/056 - Arise - Run Run Run/drums.wav',
  'other': '../DSD100/Sources/Dev/056 - Arise - Run Run Run/other.wav',
  'bass': '../DSD100/Sources/Dev/056 - Arise - Run Run Run/bass.wav'},
 50)

## The Points
- Do not use the origin mix track, using the sum of vocal and background tracks and return a new mixed track, because the old track is not ensured the normalization
### Remove vocal silence part ( slience is useless for model training)

In [6]:
def remove_silence(vocal, target_track):
    """
    This function remove slience part
    notice it's not an in place dealer
    """
    remove_list = list()
    for i, point in enumerate(vocal):
        if 10**-4 > abs(point): # be careful of the silence part shreshold
            remove_list.append(i)
    vocal_out = np.delete(vocal, remove_list)
    target_track_out = np.delete(target_track, remove_list)
    return vocal_out, target_track_out

### Extract the mix/target signals

In [7]:
def sound_tracks_extractor(file_path):
    """
    Take in a file path dictionary, return:
    (Normalized track)
    vocal track as target
    mixed track as input
    """
    signals = dict()
    for key in file_path:
        if key == 'mix':
            continue
        signals[key], _ = librosa.load(file_path.get(key), sr = PARAS.SR)
        signals[key] = librosa.util.normalize(signals.get(key))
        
    vocal_track = signals.get('vocals')
    return_vocal, _ = remove_silence(vocal_track, vocal_track)
    background_track = np.zeros(len(return_vocal))
    background_track_list = list()
    
    for key in signals:
        if 'vocals' == key:
            continue
        _, signals[key] = remove_silence(vocal_track, signals.get(key))
        background_track = signals.get(key) if not len(background_track) else background_track + signals.get(key)
        background_track_list.append(background_track)
    
    mix_track_list = [return_vocal + bg for bg in background_track_list] # this is for increase the dataset size
    
    return return_vocal, background_track_list, mix_track_list

### Generate chunks Spectrogram
#### Use MelConverter

In [8]:
b =  sound_tracks_extractor(test_path)

#### create mel_spectrogram (150 * 150 frames)

In [11]:
def frame_feature_extractor(signal, mel_converter=mel_converter):
    """
    Takes in new signals and create mel chunks 
    """
    S = mel_converter.signal_to_melspec(signal)
    if not S.shape[0] % PARAS.N_MEL == 0:
        S = S[:-1 * (S.shape[0] % PARAS.N_MEL)] # divide the mel spectrogram
        
    chunk_num = int(S.shape[0] / PARAS.N_MEL)
    mel_chunks = np.split(S, chunk_num) # create 150 * 150 data frames
    return mel_chunks, chunk_num

In [56]:
def sound_tracks_dealer(tracks_mix, mel_converter=mel_converter):
    vocal, background_track_list, mix_track_list = tracks_mix
    b_chunks, m_chunks, v_chunks = list(), list(), list()
    
    vocal_chunks, cn = frame_feature_extractor(vocal)
    for i in range(len(mix_track_list)):
        current_bg, _ = frame_feature_extractor(background_track_list[i])
        current_mix, _ = frame_feature_extractor(mix_track_list[i])
        b_chunks = b_chunks + current_bg
        m_chunks = m_chunks + current_mix
        v_chunks = v_chunks + vocal_chunks
    return v_chunks, b_chunks, m_chunks

# Build dataset

In [57]:
file_paths = dict(list(dev_file_paths.items())
                  + list(test_file_paths.items()))

## load data
Load data in to small databases

In [59]:
import time

print("start loading data...")
file_count = 100
chunk_count = 0
start_time = time.time()

for key, file_dict in file_paths.items():
    ss_time = time.time()
    count = 0
    
    '''
    vocal_signal, bg_signal, mix_signal = sound_tracks_extractor(file_dict)
    vocal_chunks, cn = frame_feature_extractor(vocal_signal)
    mix_chunks, _ = frame_feature_extractor(mix_signal)
    bg_chunks, _ = frame_feature_extractor(bg_signal)
    '''
    
    mix_tracks = sound_tracks_extractor(file_dict)
    vocal_chunks, bg_chunks, mix_chunks = sound_tracks_dealer(mix_tracks)
    cn = len(vocal_chunks)
    
    c_setpath = PARAS.DATASET_PATH + '{0}.h5'.format(key)
    c_dataset = h5py.File(c_setpath, 'a')
    
    c_dataset.create_dataset('mix', 
                       shape=(cn, PARAS.N_MEL, PARAS.N_MEL),   
                       dtype=np.float32)
    
    c_dataset.create_dataset('bg', 
                       shape=(cn, PARAS.N_MEL, PARAS.N_MEL), 
                       dtype=np.float32)

    c_dataset.create_dataset('vocal', 
                           shape=(cn, PARAS.N_MEL, PARAS.N_MEL), 
                           dtype=np.float32)
    
    for idx in range(cn):
        c_dataset['vocal'][count] = vocal_chunks[idx]
        c_dataset['mix'][count] = mix_chunks[idx]
        c_dataset['bg'][count] = bg_chunks[idx]
        count += 1
        chunk_count += 1
    
    file_count -= 1
    epoch_s = round(time.time() - ss_time, 2)
    print("=>${0}[{1}s|LEFT:{2}|CHUNK:{3}] ".format(key, epoch_s, file_count, chunk_count))
    c_dataset.close()
print('******END*******')
print("TOTAL CHUNK: {0}".format(chunk_count))
print("TOTAL TIME: {0}min".format(round(((time.time()-start_time) / 60), 2)))

start loading data...
=>$055[51.06s|LEFT:99|CHUNK:420] 
=>$084[81.17s|LEFT:98|CHUNK:912] 
=>$077[63.78s|LEFT:97|CHUNK:1494] 
=>$058[71.01s|LEFT:96|CHUNK:1938] 
=>$062[72.0s|LEFT:95|CHUNK:2505] 
=>$092[91.24s|LEFT:94|CHUNK:3333] 
=>$095[122.66s|LEFT:93|CHUNK:4143] 
=>$072[70.97s|LEFT:92|CHUNK:4686] 
=>$063[52.61s|LEFT:91|CHUNK:4818] 
=>$087[89.08s|LEFT:90|CHUNK:5082] 
=>$073[74.36s|LEFT:89|CHUNK:5760] 
=>$060[78.89s|LEFT:88|CHUNK:6318] 
=>$067[119.03s|LEFT:87|CHUNK:7083] 
=>$081[79.1s|LEFT:86|CHUNK:7416] 
=>$083[68.82s|LEFT:84|CHUNK:8631] 
=>$097[65.99s|LEFT:83|CHUNK:9213] 
=>$068[55.8s|LEFT:82|CHUNK:9687] 
=>$065[56.93s|LEFT:81|CHUNK:10101] 
=>$090[92.52s|LEFT:80|CHUNK:11055] 
=>$088[86.6s|LEFT:79|CHUNK:11835] 
=>$082[62.66s|LEFT:78|CHUNK:12201] 
=>$093[67.44s|LEFT:77|CHUNK:12609] 
=>$074[74.66s|LEFT:76|CHUNK:13164] 
=>$057[94.86s|LEFT:75|CHUNK:13788] 
=>$052[59.31s|LEFT:74|CHUNK:14355] 
=>$091[90.62s|LEFT:73|CHUNK:15024] 
=>$064[51.89s|LEFT:72|CHUNK:15492] 
=>$080[68.91s|LEFT:71|CHUNK

## Create Datasets
### Create all in one dataset

In [60]:
all_setpath = PARAS.DATASET_PATH + 'all_enhanced.h5'
all_dataset = h5py.File(all_setpath, 'a')

In [61]:

all_dataset.create_dataset('mix', 
                       shape=(chunk_count, PARAS.N_MEL, PARAS.N_MEL), 
                       dtype=np.float32)

all_dataset.create_dataset('vocal', 
                       shape=(chunk_count, PARAS.N_MEL, PARAS.N_MEL), 
                       dtype=np.float32)

all_dataset.create_dataset('bg', 
                       shape=(chunk_count, PARAS.N_MEL, PARAS.N_MEL), 
                       dtype=np.float32)

<HDF5 dataset "bg": shape (50151, 128, 128), type "<f4">

In [ ]:
count = 0
for (dirpath, dirnames, filenames) in os.walk(PARAS.DATASET_PATH):
    if filenames:
        for set_name in filenames:
            if 'h5' not in set_name or 'all' in set_name:
                continue
            set_path = PARAS.DATASET_PATH + set_name
            tmp_dataset = h5py.File(set_path, 'r')
            tmp_count = tmp_dataset['vocal'].shape[0]
            for i in range(tmp_count):
                all_dataset['mix'][count] = tmp_dataset['mix'][i]
                all_dataset['vocal'][count] = tmp_dataset['vocal'][i]
                all_dataset['bg'][count] = tmp_dataset['bg'][i]
                count += 1
            tmp_dataset.close()
all_dataset.close()
print(count)

# Split Dataset

In [63]:
import random
all_setpath = PARAS.DATASET_PATH + 'all_enhanced.h5'
all_dataset = h5py.File(all_setpath, 'r')

train_set = PARAS.DATASET_PATH+'train_enhanced.h5'
valid_set = PARAS.DATASET_PATH+'valid_enhanced.h5'
test_set = PARAS.DATASET_PATH+'test_enhanced.h5'

In [64]:
train_file = int(chunk_count * 0.8)
valid_file = int(chunk_count * 0.1)
test_file = chunk_count - train_file - valid_file
print(train_file, valid_file, test_file)

40120 5015 5016


In [65]:
files = [int(a) for a in [train_file, valid_file, test_file]]
sets = [train_set, valid_set, test_set]
print(files)

[40120, 5015, 5016]


In [66]:
idx = [i for i in range(chunk_count)]
random.seed(516)
random.shuffle(idx)

In [67]:
train_idx = idx[:files[0]]
valid_idx = idx[files[0]: files[0]+files[1]]
test_idx = idx[-files[2]:]
indices = [train_idx, valid_idx, test_idx]

In [68]:
for i, dset in enumerate(sets):
    s_set = h5py.File(dset, 'a')
    indice = indices[i]
    file_num = files[i]
    
    s_set.create_dataset('vocal', shape=(file_num, PARAS.N_MEL, PARAS.N_MEL), dtype=np.float32)
    s_set.create_dataset('mix', shape=(file_num, PARAS.N_MEL, PARAS.N_MEL), dtype=np.float32)
    s_set.create_dataset('bg', shape=(file_num, PARAS.N_MEL, PARAS.N_MEL), dtype=np.float32)
    
    count = 0
    for i in indice:
        s_set['vocal'][count] = all_dataset['vocal'][i]
        s_set['mix'][count] = all_dataset['mix'][i]
        s_set['bg'][count] = all_dataset['bg'][i]
        count += 1
        
        if count % 100 == 0:
            print('=>{0}'.format(count), end="")
    
    s_set.close()
    print()  
    print('Create Separate Datasets {0}'.format(dset))

=>100=>200=>300=>400=>500=>600=>700=>800=>900=>1000=>1100=>1200=>1300=>1400=>1500=>1600=>1700=>1800=>1900=>2000=>2100=>2200=>2300=>2400=>2500=>2600=>2700=>2800=>2900=>3000=>3100=>3200=>3300=>3400=>3500=>3600=>3700=>3800=>3900=>4000=>4100=>4200=>4300=>4400=>4500=>4600=>4700=>4800=>4900=>5000=>5100=>5200=>5300=>5400=>5500=>5600=>5700=>5800=>5900=>6000=>6100=>6200=>6300=>6400=>6500=>6600=>6700=>6800=>6900=>7000=>7100=>7200=>7300=>7400=>7500=>7600=>7700=>7800=>7900=>8000=>8100=>8200=>8300=>8400=>8500=>8600=>8700=>8800=>8900=>9000=>9100=>9200=>9300=>9400=>9500=>9600=>9700=>9800=>9900=>10000=>10100=>10200=>10300=>10400=>10500=>10600=>10700=>10800=>10900=>11000=>11100=>11200=>11300=>11400=>11500=>11600=>11700=>11800=>11900=>12000=>12100=>12200=>12300=>12400=>12500=>12600=>12700=>12800=>12900=>13000=>13100=>13200=>13300=>13400=>13500=>13600=>13700=>13800=>13900=>14000=>14100=>14200=>14300=>14400=>14500=>14600=>14700=>14800=>14900=>15000=>15100=>15200=>15300=>15400=>15500=>15600=>15700=>15800=>